### Task Generator Definitions

In [5]:
import numpy as np

class AddNTaskGenerator:
    def __init__(self, n, n_pairs=4, m_mod=10):
        self.n = n
        self.n_pairs = n_pairs
        self.m_mod = m_mod

    def generate_pair(self) -> np.ndarray:
        x = np.random.randint(0, self.m_mod)
        y = (x + self.n) % self.m_mod
        return (x, y)
    
    def generate_task(self) -> list:
        pairs = []
        for _ in range(self.n_pairs):
            while True:
                xy = self.generate_pair()
                if xy not in pairs:
                    break
            pairs.append(xy)
        return pairs

In [7]:
add3_gen = AddNTaskGenerator(3, n_pairs=5, m_mod=2**13)

add3_gen.generate_task()

[(6526, 6529), (4100, 4103), (4663, 4666), (5079, 5082), (7830, 7833)]

In [12]:
class ProjTaskGenerator:
    def __init__(self, proj, n_pairs=4):
        self.proj = proj
        self.n_pairs = n_pairs
    
    def generate_pair(self) -> np.ndarray:
        x = np.random.rand(self.proj.shape[0], self.proj.shape[0])
        y = self.proj @ x
        return (x, y)
    
    def generate_task(self) -> list:
        pairs = [self.generate_pair() for _ in range(self.n_pairs)]
        return pairs

In [ ]:
proj = np.random.rand(5, 5)
proj_gen = ProjTaskGenerator(proj, n_pairs=5)
proj_gen.generate_task()

In [21]:
class AffineTaskGenerator:
    def __init__(self, a, b, d, n_pairs=4):
        self.a = a
        self.b = b
        self.d = d
        self.n_pairs = n_pairs
    
    def generate_pair(self) -> np.ndarray:
        x = np.random.rand(self.d)
        y = self.a * x + self.b
        return (x, y)
    
    def generate_task(self) -> list:
        pairs = [self.generate_pair() for _ in range(self.n_pairs)]
        return pairs

In [ ]:
aff_gen = AffineTaskGenerator(2, 3, 5, n_pairs=4)
aff_gen.generate_task()

[(array([0.41396197, 0.1737389 , 0.13506816, 0.27361507, 0.88888621]),
  array([3.82792394, 3.34747781, 3.27013632, 3.54723015, 4.77777243])),
 (array([0.49122357, 0.87750747, 0.73392712, 0.33973896, 0.33379359]),
  array([3.98244715, 4.75501494, 4.46785424, 3.67947793, 3.66758717])),
 (array([0.23872954, 0.18730224, 0.37169544, 0.74973343, 0.7268057 ]),
  array([3.47745908, 3.37460448, 3.74339087, 4.49946686, 4.4536114 ])),
 (array([0.30929472, 0.75282629, 0.57553106, 0.60630394, 0.38589064]),
  array([3.61858944, 4.50565258, 4.15106212, 4.21260789, 3.77178128]))]

In [34]:
tasks = aff_gen.generate_task()
tensors = [np.column_stack(pair) for pair in tasks]
for tensor in tensors:
    print(tensor.shape)  # Check the shape of the generated
tensors

(5, 2)
(5, 2)
(5, 2)
(5, 2)


[array([[0.92402129, 4.84804258],
        [0.8636112 , 4.7272224 ],
        [0.53988548, 4.07977095],
        [0.04638061, 3.09276121],
        [0.38516828, 3.77033657]]),
 array([[0.72409921, 4.44819841],
        [0.2236245 , 3.447249  ],
        [0.64518727, 4.29037454],
        [0.45458604, 3.90917208],
        [0.58507326, 4.17014652]]),
 array([[0.32537193, 3.65074387],
        [0.70893743, 4.41787486],
        [0.52449021, 4.04898042],
        [0.35525496, 3.71050991],
        [0.40491259, 3.80982519]]),
 array([[0.03581688, 3.07163377],
        [0.90302215, 4.8060443 ],
        [0.66734353, 4.33468706],
        [0.68698267, 4.37396533],
        [0.27839763, 3.55679527]])]

In [36]:
pairs = np.stack(tensors, axis=0)
pairs.shape

(4, 5, 2)

### Encoder

In [28]:
import torch
import torch.nn as nn

In [54]:
D = 10
N = 4

shape_in = (D, 2)
shape_lat = (3,)

d_in = shape_in[0] * shape_in[1]
d_lat = shape_lat[0]

inp = torch.randn(N, D, 2)
print(inp.shape)  # Input shape (N, D, 2)

# Input shape (N*, D, 2)
encoder = nn.Sequential(
    nn.Flatten(),
    nn.Linear(d_in, d_in),
    nn.ReLU(),
    nn.Linear(d_in, d_in // 2),
    nn.ReLU(),
    nn.Linear(d_in // 2, d_lat)
)

encoder(inp).shape

torch.Size([4, 10, 2])


torch.Size([4, 3])

In [60]:
a, b = 2, 3

aff_gen = AffineTaskGenerator(a, b, D, n_pairs=N)
task = aff_gen.generate_task()
pairs = [np.column_stack(pair) for pair in task]
pairs = np.stack(pairs, axis=0)
pairs = torch.tensor(pairs, dtype=torch.float32)
pairs.shape  # (N, D, 2)

torch.Size([4, 10, 2])

In [61]:
inp.shape, pairs.shape

(torch.Size([4, 10, 2]), torch.Size([4, 10, 2]))

In [ ]:
z = encoder(pairs)

tensor([[-0.1542, -0.3179,  0.0792],
        [-0.1456, -0.3798,  0.0660],
        [-0.1681, -0.3738,  0.0678],
        [-0.1783, -0.3548,  0.1058]], grad_fn=<AddmmBackward0>)

In [ ]:
class LpnLite(nn.Module):
    def __init__(self, encoder, decoder):
        self.encoder = encoder
        self.decoder = decoder
        pass

    def forward(self, pairs):
        """
        Args:
            pairs. Shape (N, D, 2)
                N: # of IO pairs per program
                D: # of dimensions in input/output
                2: input/output
        """
        z = self.encoder(pairs)

        z_prime = 


            # x = torch.tensor(x, dtype=torch.float32)
            # y = torch.tensor(y, dtype=torch.float32)
            # x = x.view(1, -1)
            # y = y.view(1, -1)
        


        y = self.decoder(z)
        return y        
